In [23]:
# !pip install pandas transformers torch shap scikit-learn
# pip install datasets
# pip install transformers[torch]
# pip install 'accelerate>=0.26.0'

In [ ]:
## Deprecated todo esto

In [24]:
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sys
sys.modules.pop("transformers", None)
import transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding, 
    pipeline
)
from datasets import Dataset
import shap
import matplotlib.pyplot as plt

sys.path.append('../')


In [25]:
print(transformers.__version__)

4.51.3


In [26]:
# --- 2. Cargar y procesar datos
# Asegurate de que tu CSV tenga una columna "oracle_text"
df = pd.read_csv("../data/dataCardsclean.csv")
df = df[df["oracle_text"].notna()]  # quitar vacíos
df = df.sample(n=500)#, random_state=42)

# Heurística simple para etiquetas
palabras_funcionales = [
    "draw", "destroy", "exile", "counter", "flying", "haste", 
    "indestructible", "trample", "double strike", "lifelink", 
    "tutor", "token", "reanimate", "search", "scry"
]

def limpiar_oracle(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text.strip()

def heuristica_funcional(texto):
    texto = limpiar_oracle(texto)
    return "funcional" if any(p in texto for p in palabras_funcionales) else "no funcional"

df["label_text"] = df["oracle_text"].apply(heuristica_funcional)
df["label"] = LabelEncoder().fit_transform(df["label_text"])  # funcional=1, no funcional=0

df[["oracle_text", "label_text", "label"]].head(5)


,oracle_text,label_text,label
61559,+1: Tap target permanent. It doesn't untap dur...,funcional,0
44427,"(As this Saga enters and after your draw step,...",funcional,0
28042,Lightning Dart deals 1 damage to target creatu...,no funcional,1
79163,Vigilance\nWhenever this creature enters or at...,funcional,0
27443,Enchant green creature\nEnchanted creature get...,funcional,0


In [27]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# # Cargar el modelo y el tokenizer guardados
# model = AutoModelForSequenceClassification.from_pretrained("./modelo_guardado")
# tokenizer = AutoTokenizer.from_pretrained("./modelo_guardado")

In [28]:
from transformers import AutoModelForSequenceClassification
import gc
import copy

# Carga desde la carpeta
model_original = AutoModelForSequenceClassification.from_pretrained("../model/modelo_guardado")
tokenizer_original = AutoTokenizer.from_pretrained("../model/modelo_guardado")

# Crea una copia para modificar
model = copy.deepcopy(model_original)
tokenizer = copy.deepcopy(tokenizer_original)

# Libera el original para que no bloquee archivos
del model_original
del tokenizer_original
gc.collect()

203

In [29]:
# --- 3. Preparar tokenizer y datasets
#tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("../model/modelo_guardado")

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["oracle_text"], df["label"], test_size=0.2, random_state=42
)

train_enc = tokenizer(list(train_texts), truncation=True, padding=True)
val_enc = tokenizer(list(val_texts), truncation=True, padding=True)

train_dataset = Dataset.from_dict({
    "input_ids": train_enc["input_ids"],
    "attention_mask": train_enc["attention_mask"],
    "labels": list(train_labels)
})

val_dataset = Dataset.from_dict({
    "input_ids": val_enc["input_ids"],
    "attention_mask": val_enc["attention_mask"],
    "labels": list(val_labels)
})


In [30]:
# --- 4. Entrenar modelo BERT
#model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("../model/modelo_guardado")

training_args = TrainingArguments(
    output_dir="./resultados",
    # evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir="./logs",
    save_strategy="no",
    load_best_model_at_end=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

trainer.train()


C:\Users\Edurne\AppData\Local\Temp\ipykernel_7596\983852042.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\Edurne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=50, training_loss=0.13483617782592774, metrics={'train_runtime': 187.8715, 'train_samples_per_second': 2.129, 'train_steps_per_second': 0.266, 'total_flos': 14178151262400.0, 'train_loss': 0.13483617782592774, 'epoch': 1.0})

In [31]:
# --- 5. Explicación con SHAP (¡la parte divertida!)
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

# Seleccionar textos para explicar
ejemplos = df["oracle_text"].sample(3, random_state=42).tolist()

# Crear y visualizar explicaciones
explainer = shap.Explainer(pipe)
shap_values = explainer(ejemplos)

# Visualizar una predicción explicada
shap.plots.text(shap_values[0])


Device set to use cpu


c:\Users\Edurne\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
PartitionExplainer explainer: 4it [01:25, 28.50s/it]                       


In [32]:
model.save_pretrained("../model/modelo_guardado")  # Guarda el modelo
tokenizer.save_pretrained("../model/modelo_guardado")  # Guarda el tokenizer

('../model/modelo_guardado\\tokenizer_config.json',
 '../model/modelo_guardado\\special_tokens_map.json',
 '../model/modelo_guardado\\vocab.txt',
 '../model/modelo_guardado\\added_tokens.json',
 '../model/modelo_guardado\\tokenizer.json')

In [33]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# # Cargar el modelo y el tokenizer guardados
# model = AutoModelForSequenceClassification.from_pretrained("./modelo_guardado")
# tokenizer = AutoTokenizer.from_pretrained("./modelo_guardado")

In [ ]:
#Esto tarda una hora, no tocar.

# from torch.nn.functional import softmax
# import torch

# model.eval()
# label_map = {0: 0, 1: 1}

# def predecir_funcionalidad(texto):
#     inputs = tokenizer(texto, return_tensors="pt", truncation=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         probs = softmax(outputs.logits, dim=-1)
#         clase = torch.argmax(probs, dim=1).item()
#         return label_map[clase]

# df = pd.read_csv("../data/dataCardsclean.csv")

# df["funcional"] = df["oracle_text"].apply(predecir_funcionalidad)

In [39]:
df

,name,cmc,oracle_text,reserved,game_changer,foil,nonfoil,promo,reprint,textless,...,legal_commander,legal_pauper,keyword_count,has_evasion,mana_produced_count,power_num,toughness_num,rarity_labeled,final_price_eur,funcional
0,Fury Sliver,6.0,All Sliver creatures have double strike.,False,False,True,True,False,False,False,...,1,0,0,0,0,3.0,3.0,1,0.21,1
1,Kor Outfitter,2.0,"When this creature enters, you may attach targ...",False,False,True,True,False,False,False,...,1,1,0,0,0,2.0,2.0,0,0.18,1
2,Siren Lookout,3.0,"Flying\nWhen this creature enters, it explores...",False,False,True,True,False,False,False,...,1,1,2,1,0,1.0,2.0,0,0.09,0
3,Web,1.0,Enchant creature (Target a creature as you cas...,False,False,False,True,False,True,False,...,1,0,1,0,0,-2.0,-2.0,2,1.03,0
4,Wastewood Verge,0.0,{T}: Add {G}.\n{T}: Add {B}. Activate only if ...,False,False,True,True,True,True,False,...,1,0,0,0,2,-2.0,-2.0,2,5.92,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85130,Aggressive Biomancy,2.0,Create X tokens that are copies of target crea...,False,False,False,True,False,False,False,...,1,0,1,0,0,-2.0,-2.0,2,0.41,0
85131,Horned Troll,3.0,{G}: Regenerate this creature.,False,False,False,True,False,True,False,...,1,1,0,0,0,2.0,2.0,0,0.18,1
85132,Faerie Bladecrafter,3.0,Flying\nWhenever one or more Faeries you contr...,False,False,False,True,False,False,False,...,1,0,1,1,0,2.0,2.0,2,0.50,0
85133,Exultant Skymarcher,3.0,Flying,False,False,True,True,False,False,False,...,1,1,1,1,0,2.0,3.0,0,0.09,0
